In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
import matplotlib.pylab as plt
from os import listdir
import pandas as pd
from PIL import Image

In [2]:
hists = {}
path = './daily_stock_data/'
for file in [f for f in listdir(path) if f.endswith('.csv')]:
    try:
        hists[file[:-4]] = pd.read_csv(path+file, index_col='timestamp')
    except:
        try:
            hists[file[:-4]] = pd.read_csv(path+file, index_col='time')
        except:
            pass

In [3]:
file = 'stock_graphs/ACAC_2021-02-19/ACAC_2021-02-19_100.jpeg'
image = Image.open(file)

In [68]:
image.save('test.jpeg')

In [4]:
from matplotlib import image
from matplotlib import pyplot
# load image as pixel array
data = image.imread(file)

In [98]:
train_x = []
train_y = []
for p in ['buy', 'sell', 'nothing']:
    path = f'stock_graphs/train/{p}/'
    for file in [f for f in listdir(path)]:
        train_x.append(image.imread(path+file))
        train_y.append(p)

In [99]:
test_x = []
test_y = []
for p in ['buy', 'sell', 'nothing']:
    path = f'stock_graphs/test/{p}/'
    for file in [f for f in listdir(path)]:
        test_x.append(image.imread(path+file))
        test_y.append(p)

In [59]:
## from keras import backend as K
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image_dataset_from_directory

In [35]:
datagen=keras.preprocessing.image.ImageDataGenerator(data_format=K.image_data_format())

In [38]:
batch_size_train = 20
batch_size_val = 10
batch_size_test = 25
num_classes= 3
intereseted_folder='Documents'
STANDARD_SIZE=(862, 1200)

In [40]:
train_path = './stock_graphs/'
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224),
                                                         classes=['sell', 'buy', 'nothing'], 
                                                         batch_size=batch_size_train)
type(train_batches)

Found 0 images belonging to 3 classes.


tensorflow.python.keras.preprocessing.image.DirectoryIterator

In [52]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=0,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

# data = image.imread(file)
# x = data.reshape(1, data.shape[2], data.shape[0], data.shape[1])
img = load_img(file)  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
# x = x.reshape(1, x.shape[2], x.shape[0], x.shape[1])

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='test', save_prefix='testing', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [53]:
x.shape[1:]

(862, 1200, 3)

In [69]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=x.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

In [70]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [71]:
batch_size = 16
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'stock_graphs/train',  # this is the target directory
        target_size=x.shape[1:-1],  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        './stock_graphs/test',
        target_size=x.shape[1:-1],
        batch_size=batch_size,
        class_mode='binary')

Found 7577 images belonging to 4 classes.
Found 1758 images belonging to 4 classes.


In [72]:
train_generator = image_dataset_from_directory(
        'stock_graphs/train',  # this is the target directory
        image_size=x.shape[1:-1],  # all images will be resized to 150x150
        batch_size=batch_size,)  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = image_dataset_from_directory(
        './stock_graphs/test',
        image_size=x.shape[1:-1],
        batch_size=batch_size,)

Found 7577 files belonging to 4 classes.
Found 1758 files belonging to 4 classes.


In [73]:
model.fit_generator(
    train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save_weights('first_try.h5')  # always save your weights after training or during training

Epoch 1/50
125/125 [==============================] - 955s 8s/step - loss: -222922035727.1935 - accuracy: 0.0341 - val_loss: -4310316351488.0000 - val_accuracy: 0.0225
Epoch 2/50
125/125 [==============================] - 912s 7s/step - loss: -12031094365297.7773 - accuracy: 0.0246 - val_loss: -59563701174272.0000 - val_accuracy: 0.0225
Epoch 3/50
125/125 [==============================] - 924s 7s/step - loss: -103949933573591.3594 - accuracy: 0.0219 - val_loss: -303850816798720.0000 - val_accuracy: 0.0225
Epoch 4/50
125/125 [==============================] - 744s 6s/step - loss: -436980047883010.0625 - accuracy: 0.0263 - val_loss: -807042071658496.0000 - val_accuracy: 0.0250


In [75]:
model.predict(validation_generator)

c:\users\mcian\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [93]:
t.sum()

1758.0

In [94]:
model_first = model

In [105]:
import numpy as np

In [106]:
np.array(train_x).shape

MemoryError: Unable to allocate 21.9 GiB for an array with shape (7577, 862, 1200, 3) and data type uint8

In [107]:
x.shape[1:]

(862, 1200, 3)

In [114]:
import gc; gc.collect()

40

In [110]:
del model_first

In [109]:
# CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=x.shape[1:]))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy', 'mae', 'mse'])

ResourceExhaustedError: OOM when allocating tensor with shape[16418688,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:RandomUniform]

In [67]:
input_shape = x.shape[1:]

In [68]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])


class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

history = AccuracyHistory()

# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(x_test, y_test),
#           callbacks=[history])
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])
# plt.plot(range(1, 11), history.acc)
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.show()

ResourceExhaustedError: OOM when allocating tensor with shape[4029696,1000] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:Mul]

In [ ]:
model.fit_generator(
    train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save_weights('first_try.h5')  # always save your weights after training or during training